### Домашнее задание <a class="anchor" id="hw"></a><center>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

__Задача:__ подойдет ли репетитор для подготовки к экзамену по математике

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def standard_scale(X):
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    return (X - mean) / std

In [4]:
X_st = X.copy().astype(np.float64)
X_st = X_st.T
X_st[:, 1:4] = standard_scale(X_st[:, 1:4])
X_st

array([[ 1.        , -0.70710678, -0.97958969, -0.89625816],
       [ 1.        , -0.70710678, -0.56713087, -0.89625816],
       [ 1.        , -0.35355339, -0.46401617,  0.38411064],
       [ 1.        ,  0.70710678, -0.77336028, -0.89625816],
       [ 1.        ,  0.        ,  0.97958969,  0.38411064],
       [ 1.        , -1.06066017, -0.36090146, -0.89625816],
       [ 1.        ,  0.70710678,  1.08270439,  1.66447944],
       [ 1.        ,  2.47487373,  2.11385144,  1.66447944],
       [ 1.        , -0.70710678, -1.08270439, -0.89625816],
       [ 1.        , -0.35355339,  0.05155735,  0.38411064]])

**1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.**

In [5]:
def calc_logloss(y, y_pred):
    eps = 1e-8
    y_pred = np.clip(y_pred, eps, 1 - eps)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [6]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0])
calc_logloss(y1, y_pred1)

0.052680262828913194

**2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.**

In [7]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [8]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
#         if i % (iterations / 10) == 0:
#             print(i, W, err)
            
    final_error = calc_logloss(y, y_pred)
    return W, final_error

In [9]:
iterations = np.logspace(2, 4, 4, dtype=np.int)
etas = np.linspace(1e-4, 8, 40)

best_error = np.inf
best_params = {}

for iteration in iterations:
    for eta in etas:
        W, error = eval_model(X_st, y, iterations=iteration, eta=eta)
        if error < best_error:
            best_error = error
            best_params = {
                'iteration': iteration,
                'eta': eta
            }

print(f'Ошибка {best_error} при параметрах {best_params}')

Ошибка 0.023317887440989456 при параметрах {'iteration': 10000, 'eta': 8.0}


**3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).**

In [10]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba

In [11]:
calc_pred_proba(W, X_st)

array([1.08249661e-01, 4.60778894e-09, 1.00000000e+00, 1.70995272e-23,
       9.92262610e-01, 3.07865257e-08, 1.00000000e+00, 1.97849542e-03,
       8.96872439e-01, 1.00000000e+00])

**4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).**

In [12]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [13]:
pred = calc_pred(W, X_st)
pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

**5. Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.**

*Accuracy*

In [14]:
def accuracy(y, y_pred):
    accuracy = np.mean(y == y_pred)
    return accuracy

In [15]:
print(f'True \t  {y}')
print(f'Predicted {pred}')

True 	  [0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]
Predicted [0 0 1 0 1 0 1 0 1 1]


In [16]:
accuracy(y, pred)

1.0

*Confusion Matrix*

In [17]:
def confusion_matrix(y, y_pred):
    cm = np.zeros((2, 2))
    for i in range(len(y)): 
        if y[i] == y_pred[i] == 1: # TP
            cm[0][0] += 1
        elif y[i] == y_pred[i] == 0: # TN
            cm[1][1] += 1
        elif y[i] != y_pred[i] and y[i] == 1: # FN
            cm[1][0] += 1
        elif y[i] != y_pred[i] and y[i] == 0: # FP
            cm[0][1] += 1
    return cm

In [18]:
cm = confusion_matrix(y, pred)
cm

array([[5., 0.],
       [0., 5.]])

*Precision*

In [19]:
def precision(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    precision = TP / (TP + FP)
    return precision

In [20]:
precision(y, pred)

1.0

*Recall*

In [21]:
def recall(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    recall = TP / (TP + FN)
    return recall

In [22]:
recall(y, pred)

1.0

*F-score*

In [23]:
def f_score(y, y_pred):
    pr = precision(y, y_pred)
    rec = recall(y, y_pred)
    f_score = 2 * pr * rec / (pr + rec)
    return f_score

In [24]:
f_score(y, pred)

1.0

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


print(f'accuracy {accuracy_score(y, pred)}')
print(f'precision {precision_score(y, pred)}')
print(f'recall {recall_score(y, pred)}')
print(f'f-score {f1_score(y, pred)}')
print(f'\n {confusion_matrix(y, pred)}')

accuracy 1.0
precision 1.0
recall 1.0
f-score 1.0

 [[5 0]
 [0 5]]


**6. Могла ли модель переобучиться? Почему?**

В нашей задаче модель не могла переобучиться. Как правило, к переобучению ведет выбор слишком сложной модели (на примере полиномиальной регрессии - слишком большая степень полинома), а в данном случае модель линейна.  
Если факт переобучения всё же будет выявлен, рекомендуется увеличить количество данных в наборе - это единственное слабое место модели.